In [1]:
# Import our libraries 

# Pandas and numpy for data wrangling
import pandas as pd
import numpy as np

# Seaborn / matplotlib for visualization 
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

# Helper function to split our data
from sklearn.model_selection import train_test_split

# Helper fuctions to evaluate our model.
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report, roc_auc_score, f1_score

# Import our Decision Tree
from sklearn.tree import DecisionTreeClassifier 

# Import our Random Forest 
from sklearn.ensemble import RandomForestClassifier

# Import the trees from sklearn
from sklearn import tree

# Helper functions to visualize our trees
from sklearn.tree import plot_tree, export_text

#tree regression
from sklearn.ensemble import RandomForestRegressor


In [2]:
df = pd.read_csv('../dataset/encoded_dataset_nov_17.csv')
df.head()

,Type,Breed,Color,Sex,Size,Animal ID,Intake Date,Outcome Date,Days in Shelter,Intake Type,Outcome Type,Intake Condition,Outcome Condition,Age,Color_encoded,Sex_encoded,Intake Condition_encoded,Intake Type_encoded,Breed_encoded,Size_encoded
0,CAT,DOMESTIC LH,BLACK/WHITE,Neutered,SMALL,A420773,09/29/2023,09/30/2023,1,STRAY,RETURN TO OWNER,HEALTHY,HEALTHY,10,1,2,0,5,8,3
1,CAT,DOMESTIC SH,ORG TABBY/WHITE,Spayed,KITTN,A417889,05/30/2023,08/08/2023,70,STRAY,ADOPTION,UNKNOWN,HEALTHY,0,22,3,3,5,13,0
2,CAT,DOMESTIC SH,BLACK,Neutered,KITTN,A418221,06/13/2023,08/08/2023,56,STRAY,ADOPTION,UNKNOWN,HEALTHY,0,0,2,3,5,13,0
3,CAT,SIAMESE/MIX,LYNX PT,Neutered,KITTN,A420264,09/12/2023,09/30/2023,18,OWNER SURRENDER,ADOPTION,HEALTHY,HEALTHY,0,18,2,0,3,34,0
4,CAT,DOMESTIC SH,BLACK,Neutered,KITTN,A419416,08/01/2023,09/30/2023,60,STRAY,ADOPTION,UNKNOWN,HEALTHY,0,0,2,3,5,13,0


## We are predicting if a cat will stay more than a month in the shelter (yes = 1, no =0) ☟

In [6]:
(df['Days in Shelter']>60).sum()

423

In [7]:
(df['Days in Shelter']<60).sum()

4305

In [8]:
# this part is to predict long or short stay (0,1)
import pandas as pd

def categorize_stay(days_stayed):
    if days_stayed <= 60:
        return 0
    else:
        return 1

# Example usage:
# Assuming df is your DataFrame and "days_stayed" is the column you want to categorize
df['stay'] = df['Days in Shelter'].apply(categorize_stay)
df.tail()

,Type,Breed,Color,Sex,Size,Animal ID,Intake Date,Outcome Date,Days in Shelter,Intake Type,...,Intake Condition,Outcome Condition,Age,Color_encoded,Sex_encoded,Intake Condition_encoded,Intake Type_encoded,Breed_encoded,Size_encoded,stay
4744,CAT,DOMESTIC SH,BLACK,Neutered,MED,A319435,02/03/2016,02/19/2016,16,OWNER SURRENDER,...,HEALTHY,HEALTHY,9,0,2,0,3,13,2,0
4745,CAT,DOMESTIC SH,BRN TABBY/WHITE,Spayed,KITTN,A413165,09/15/2022,10/29/2022,44,OWNER SURRENDER,...,HEALTHY,HEALTHY,1,8,3,0,3,13,0,0
4746,CAT,DOMESTIC SH,WHITE/BLACK,Neutered,KITTN,A389577,07/20/2019,07/30/2019,10,TRANSFER,...,HEALTHY,HEALTHY,4,26,2,0,6,13,0,0
4747,CAT,DOMESTIC SH,BRN TABBY/WHITE,Neutered,SMALL,A310168,08/14/2014,08/23/2014,9,STRAY,...,HEALTHY,HEALTHY,9,8,2,0,5,13,3,0
4748,CAT,DOMESTIC SH,BLUE CREAM,Spayed,KITTN,A412870,08/16/2022,09/29/2022,44,STRAY,...,HEALTHY,HEALTHY,1,4,3,0,5,13,0,0


In [9]:
df.head()

,Type,Breed,Color,Sex,Size,Animal ID,Intake Date,Outcome Date,Days in Shelter,Intake Type,...,Intake Condition,Outcome Condition,Age,Color_encoded,Sex_encoded,Intake Condition_encoded,Intake Type_encoded,Breed_encoded,Size_encoded,stay
0,CAT,DOMESTIC LH,BLACK/WHITE,Neutered,SMALL,A420773,09/29/2023,09/30/2023,1,STRAY,...,HEALTHY,HEALTHY,10,1,2,0,5,8,3,0
1,CAT,DOMESTIC SH,ORG TABBY/WHITE,Spayed,KITTN,A417889,05/30/2023,08/08/2023,70,STRAY,...,UNKNOWN,HEALTHY,0,22,3,3,5,13,0,1
2,CAT,DOMESTIC SH,BLACK,Neutered,KITTN,A418221,06/13/2023,08/08/2023,56,STRAY,...,UNKNOWN,HEALTHY,0,0,2,3,5,13,0,0
3,CAT,SIAMESE/MIX,LYNX PT,Neutered,KITTN,A420264,09/12/2023,09/30/2023,18,OWNER SURRENDER,...,HEALTHY,HEALTHY,0,18,2,0,3,34,0,0
4,CAT,DOMESTIC SH,BLACK,Neutered,KITTN,A419416,08/01/2023,09/30/2023,60,STRAY,...,UNKNOWN,HEALTHY,0,0,2,3,5,13,0,0


In [10]:
count_ones = df['stay'].value_counts().get(1)
print(f"Number of occurrences of 1: {count_ones}")

count_zeros = df['stay'].value_counts().get(0)
print(f"Number of occurrences of 0: {count_zeros}")


Number of occurrences of 1: 423
Number of occurrences of 0: 4326


In [8]:
independent_variables = ['Color_encoded', 'Sex_encoded',  
                     'Intake Condition_encoded', 'Intake Type_encoded']

dependent_variable = 'stay'

In [9]:
X= df[independent_variables]
y= df['stay']
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) 
X_train.shape, X_test.shape

((2236, 4), (746, 4))

In [10]:
(y_test==1).sum()

409

In [11]:
(y_test==0).sum()

337

In [12]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
params = { 
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'max_leaf_nodes': [5, 10]
}
grid_search_cv = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=params,
    scoring='accuracy',
    cv=5, 
    verbose=1
)

# 1. Fit your GridSearchCV with your training data. 
grid_search_cv.fit(X_train, y_train)

# Print the best parameters it found
print(grid_search_cv.best_estimator_) 

Fitting 5 folds for each of 162 candidates, totalling 810 fits


KeyboardInterrupt: 

In [ ]:
model = grid_search_cv.best_estimator_


# Now lets evaluate our model
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy Score: %f" % accuracy)

precision = precision_score(y_true=y_test, y_pred= y_pred)
print("Precision Score: %f" % precision)

recall = recall_score(y_true=y_test, y_pred= y_pred)
print("Recall Score: %f" % recall)

f1 = f1_score(y_true=y_test, y_pred= y_pred)
print('F1 Score: %f' % f1)

# Calculate predicted probabilities, keep only probability for when class = 1
y_pred_proba = model.predict_proba(X_test)[:,1]

auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)
print('AUC Score: %f' % auc)

In [ ]:
ratio = (y_test == 1).sum() / ((y_test == 0) | (y_test == 1)).sum()
ratio

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming y_test and y_pred are already defined

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm = cm.round(2)

# Extract individual elements from the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 5))

# Plot the heatmap
sns.heatmap(cm, annot=True, cmap='Greens', fmt='g', cbar=False, ax=ax)

# Label the heatmap with TN, FP, FN, and TP values
ax.text(0, 0, f'TN: {tn}', ha='center', va='center', color='blue', fontsize=12)
ax.text(0, 1, f'FP: {fp}', ha='center', va='center', color='red', fontsize=12)
ax.text(1, 0, f'FN: {fn}', ha='center', va='center', color='red', fontsize=12)
ax.text(1, 1, f'TP: {tp}', ha='center', va='center', color='blue', fontsize=12)

# Set labels for x and y axes
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Display the plot
plt.show()


In [ ]:
# print(y_test.tolist())

# ⬆︎⬆︎-----------  F1 Score: 0.677930 -------- ⬆︎⬆︎ (best so far)

## in order to solve the imbalance in calss we use the below code ☟

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Load your dataset
df2 = pd.read_csv('../dataset/df_only_numeric_value_nov_05.csv')

# Define the function to categorize stay
def categorize_stay(days_stayed):
    if days_stayed <= 30:
        return 0
    else:
        return 1

# Apply the categorization
df2['stay'] = df2['days_stayed'].apply(categorize_stay)

independent_variables = ['Primary_Color_encoded', 'Sex_encoded',  
                     'Intake_Condition_encoded', 'Intake_Type_encoded']

dependent_variable = 'stay'

X = df2[independent_variables]
y = df2['stay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) 

# Use imbalanced-learn to resample the training data
# ros = RandomOverSampler(sampling_strategy='auto', random_state=42)  # You can adjust the sampling_strategy as needed
# X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Alternatively, you can use RandomUnderSampler for undersampling
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# Define the hyperparameter grid for GridSearchCV
params = { 
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'max_leaf_nodes': [5, 10]
}

# Use GridSearchCV with RandomForestClassifier
grid_search_cv = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=params,
    scoring='accuracy',
    cv=5, 
    verbose=1
)

# Fit the model on the resampled data
grid_search_cv.fit(X_resampled, y_resampled)

# Print the best parameters it found
print(grid_search_cv.best_estimator_) 

# Extract the best model from GridSearchCV
model = grid_search_cv.best_estimator_

# Evaluate the model on the original test set
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy Score: %f" % accuracy)

precision = precision_score(y_true=y_test, y_pred=y_pred)
print("Precision Score: %f" % precision)

recall = recall_score(y_true=y_test, y_pred=y_pred)
print("Recall Score: %f" % recall)

f1 = f1_score(y_true=y_test, y_pred=y_pred)
print('F1 Score: %f' % f1)

# Calculate predicted probabilities, keep only probability for when class = 1
y_pred_proba = model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)
print('AUC Score: %f' % auc)


In [ ]:
ratio = (y_test == 1).sum() / ((y_test == 0) | (y_test == 1)).sum()
ratio

In [ ]:
(y_test == 1).sum()

In [ ]:

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm = cm.round(2)

# Extract individual elements from the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 5))

# Plot the heatmap
sns.heatmap(cm, annot=True, cmap='Greens', fmt='g', cbar=False, ax=ax)

# Label the heatmap with TN, FP, FN, and TP values
ax.text(0, 0, f'TN: {tn}', ha='center', va='center', color='blue', fontsize=12)
ax.text(0, 1, f'FP: {fp}', ha='center', va='center', color='red', fontsize=12)
ax.text(1, 0, f'FN: {fn}', ha='center', va='center', color='red', fontsize=12)
ax.text(1, 1, f'TP: {tp}', ha='center', va='center', color='blue', fontsize=12)

# Set labels for x and y axes
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Display the plot
plt.show()


# ⬆︎⬆︎-------  F1 Score: 0.630460 ------- ⬆︎⬆︎(2nd best so far)




### Now we will try to predict how many month a cat will stay using regression ☟

In [ ]:
df_month = pd.read_csv('../dataset/df_only_numeric_value_nov_05.csv')
df_month.head()

In [ ]:
# this part is to predict month stay
import pandas as pd

def categorize_days(days_stayed):
    if days_stayed <= 15:
        return 0
    elif days_stayed <= 45:
        return 1
    elif days_stayed <= 75:
        return 2
    elif days_stayed <= 105:
        return 3
    # Add more conditions as needed up to 1200 days
    elif days_stayed <= 1200:
        # Calculate the month based on the provided conditions
        return (days_stayed - 105) // 30 + 3
#     else:
#         # Handle cases beyond 1200 days if needed
#         return 100  # or any other value to indicate an outlier

# Example usage:
# Assuming df is your DataFrame and "days_stayed" is the column you want to categorize
df_month['months_stayed'] = df_month['days_stayed'].apply(categorize_days)
df_month.head()


In [ ]:
selected_features = ['Primary_Color_encoded', 'Sex_encoded',  
                     'Intake_Condition_encoded', 'Intake_Type_encoded']

# dependent_variable = 'days_stayed'
X= df_month[selected_features]
y= df_month['months_stayed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

params = { 
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'max_leaf_nodes': [5, 10]
}

# Initialize your GridSearchCV with a RandomForestRegressor, your param_grid, and what you are optimizing for (MSE).
grid_search_cv = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=params,
    scoring='neg_mean_squared_error',  # For regression, use neg_mean_squared_error as the scoring metric.
    cv=5, 
    verbose=1
)

# Fit your GridSearchCV with your training data.
grid_search_cv.fit(X_train, y_train)

# Get the best estimator based on the mean squared error.
best_model = grid_search_cv.best_estimator_


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score  # Import r2_score

# # Map 'class' to binary labels (0 for <=50K and 1 for >50K)
# y_binary = (y == ' >50K').astype(int)

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize the Decision Tree classifier
# model = RandomForestRegressor(random_state=42)

# Fit the model
best_model.fit(X_train, y_train)

# Make predictions

y_pred = np.round(best_model.predict(X_test))

# Specify the positive label

# Evaluate the model
mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
print("Mean Squared Error: %f" % mse)

r2 = r2_score(y_true=y_test, y_pred=y_pred)
print("R-squared Score: %f" % r2)

## Eval Model 
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy Score: %f" % accuracy)

In [ ]:
len(y_pred)
# max(y_pred)

In [ ]:
y_pred

In [ ]:
# y_pred_series is our Pandas Series
y_pred_series = pd.Series(y_pred)

# Count occurrences of each unique value
value_counts = y_pred_series.value_counts()

# Sort the values based on unique values
sorted_value_counts = value_counts.sort_index()

# Display the sorted result
print("Sorted Value Counts:")
print(sorted_value_counts)


In [ ]:
len(y_test)

In [ ]:

# y_test_series is our Pandas Series
y_test_series = pd.Series(y_test)

# Count occurrences of each unique value
value_counts = y_test_series.value_counts()

# Sort the values based on unique values
sorted_value_counts = value_counts.sort_index()

# Display the sorted result
print("Sorted Value Counts:")
print(sorted_value_counts)


In [ ]:
print(y_test.tolist())


# ------ Accuracy Score: 0.180905 --------- (4th best so far)

# Trying with Randomclassifier

In [ ]:
df_month_short = pd.read_csv('../dataset/df_only_numeric_value_nov_05.csv')
df_month_short.head()

In [ ]:
# this part is to predict month stay
import pandas as pd

def categorize_days(days_stayed):
    if days_stayed <= 285:
        return np.ceil((days_stayed-15 )/30).astype(int)
    else:
        # Handle cases beyond 1200 days if needed
        return 99  # or any other value to indicate an outlier

# Example usage:
# Assuming df is your DataFrame and "days_stayed" is the column you want to categorize
df_month_short['months_stayed'] = df_month_short['days_stayed'].apply(categorize_days)
df_month_short.head()


In [ ]:
# counting each groups frequency
grouped_data = df_month_short['months_stayed'].value_counts().sort_index()
grouped_data.columns = ['months_stayed', 'Count']
print(grouped_data)

In [ ]:
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight

In [ ]:
selected_features = ['Primary_Color_encoded', 'Sex_encoded',  
                     'Intake_Condition_encoded', 'Intake_Type_encoded']

# dependent_variable = 'days_stayed'
X= df_month[selected_features]
y= df_month['months_stayed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = { 
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'max_leaf_nodes': [5, 10]
}

# Initialize your GridSearchCV with a RandomForestRegressor, your param_grid, and what you are optimizing for (MSE).
grid_search_cv = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=params,
    scoring='neg_mean_squared_error',  # For regression, use neg_mean_squared_error as the scoring metric.
    cv=5, 
    verbose=1
)

# Fit your GridSearchCV with your training data.
grid_search_cv.fit(X_train, y_train)

# Get the best estimator based on the mean squared error.
best_model = grid_search_cv.best_estimator_


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score  # Import r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the model
best_model.fit(X_train, y_train)

# Make predictions

y_pred = best_model.predict(X_test)

# Calculate precision, recall, and F1 score

precision = precision_score(y_true=y_test, y_pred=y_pred, average='weighted')  # Adjust average if needed
print("Precision Score: %f" % precision)

recall = recall_score(y_true=y_test, y_pred=y_pred, average='weighted')  # Adjust average if needed
print("Recall Score: %f" % recall)

f1 = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')  # Adjust average if needed
print('F1 Score: %f' % f1)

# Calculate predicted probabilities, keep only probability for when class = 1
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]


# --------- F1 Score: 0.260524 ------------ (3rd best so far)

# 2nd approch (all code together) RandomForestRegression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from imblearn.under_sampling import RandomUnderSampler

# Load the dataset
df_month_short = pd.read_csv('../dataset/df_only_numeric_value_nov_05.csv')

# Define a function to categorize days
def categorize_days(days_stayed):
    if days_stayed <= 285:
        return np.ceil((days_stayed - 15) / 30).astype(int)
    else:
        return 99  # or any other value to indicate an outlier

# Apply the categorization to create a new column 'months_stayed'
df_month_short['months_stayed'] = df_month_short['days_stayed'].apply(categorize_days)

# Define selected features
selected_features = ['Primary_Color_encoded', 'Sex_encoded',
                     'Intake_Condition_encoded', 'Intake_Type_encoded']

# Set up X and y
X = df_month_short[selected_features]
y = df_month_short['months_stayed']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle class imbalance using RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# Define the parameter grid for GridSearchCV
params = {
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'max_leaf_nodes': [5, 10]
}

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_cv = GridSearchCV(
    estimator=rf_classifier,
    param_grid=params,
    scoring='accuracy',  # Use accuracy for classification
    cv=5,
    verbose=1
)

# Fit GridSearchCV with resampled training data
grid_search_cv.fit(X_resampled, y_resampled)

# Get the best estimator based on accuracy
best_model = grid_search_cv.best_estimator_

# Fit the best model with resampled training data
best_model.fit(X_resampled, y_resampled)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy Score: %f" % accuracy)

f1 = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
print("F1 Score: %f" % f1)


In [ ]:
len(y_pred)
# max(y_pred)

In [ ]:
y_pred

In [ ]:
# y_pred_series is our Pandas Series
y_pred_series = pd.Series(y_pred)

# Count occurrences of each unique value
value_counts = y_pred_series.value_counts()

# Sort the values based on unique values
sorted_value_counts = value_counts.sort_index()

# Display the sorted result
print("Sorted Value Counts:")
print(sorted_value_counts)


In [ ]:
len(y_test)

In [ ]:

# y_test_series is our Pandas Series
y_test_series = pd.Series(y_test)

# Count occurrences of each unique value
value_counts = y_test_series.value_counts()

# Sort the values based on unique values
sorted_value_counts = value_counts.sort_index()

# Display the sorted result
print("Sorted Value Counts:")
print(sorted_value_counts)


In [ ]:
print(y_test.tolist())


# --------- F1 Score: 0.142566 ------------ (worst so far)

# The below part has error ☟

In [ ]:
# X = df[independent_variables]
# y = df[dependent_variable]

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# # Parameter grid for GridSearchCV
# params = { 
#     'n_estimators': [50, 100, 150],
#     'max_features': ['sqrt', 'log2', None],
#     'max_depth': [None, 5, 10],
#     'min_samples_split': [2, 5, 10],
#     'max_leaf_nodes': [5, 10]
# }

# # Initialize GridSearchCV with RandomForestClassifier
# grid_search_cv = GridSearchCV(
#     estimator=RandomForestClassifier(random_state=42),
#     param_grid=params,
#     scoring='accuracy',
#     cv=5, 
#     verbose=1
# )

# # Fit GridSearchCV with training data
# grid_search_cv.fit(X_train, y_train)

# # Get the best estimator based on accuracy
# best_model = grid_search_cv.best_estimator_

# # Make predictions on the test set
# y_pred = best_model.predict(X_test)

# # Evaluate the best model
# accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
# precision = precision_score(y_true=y_test, y_pred=y_pred, zero_division=1)
# recall = recall_score(y_true=y_test, y_pred=y_pred, zero_division=1)
# f1 = f1_score(y_true=y_test, y_pred=y_pred, zero_division=1)

# print("Accuracy Score: %f" % accuracy)
# print("Precision Score: %f" % precision)
# print("Recall Score: %f" % recall)
# print('F1 Score: %f' % f1)

# # Calculate predicted probabilities for the positive class
# y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# # Calculate AUC score
# auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)
# print('AUC Score: %f' % auc)



# The below part I kept for only reference purposes, no use here at this moment ☟

In [ ]:

# model = RandomForestClassifier()
# X = df[independent_variables]
# y = df[dependent_variable]

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# # Fit the model to the training data
# model.fit(X_train, y_train)

# # Now let's evaluate our model on the test set
# y_pred = model.predict(X_test)

# model.fit(X_train, y_train)

# # Now lets evaluate our model
# y_pred = model.predict(X_test)


# ## Eval Model 
# accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
# print("Accuracy Score: %f" % accuracy)

# precision = precision_score(y_true=y_test, y_pred=y_pred)
# print("Precision Score: %f" % precision)

# recall = recall_score(y_true=y_test, y_pred=y_pred)
# print("Recall Score: %f" % recall)

# f1 = f1_score(y_true=y_test, y_pred=y_pred)
# print('F1 Score: %f' % f1)

# # Calculate predicted probabilities
# y_pred_proba = model.predict_proba(X_test)

# # # Keep only the proba for True
# y_pred_proba = y_pred_proba[:,1]

# # Compute auc score
# auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)
# print('AUC Score: %f' % auc)


# # Produce classification Report
# print(classification_report(y_test, y_pred))


## ----------------------ignore the below part for now -------------------------

In [ ]:
# model = OneVsRestClassifier(DecisionTreeClassifier(max_depth=2))

# X = df[independent_variables]
# y = df[dependent_variable]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# # Eval Model
# accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
# print("Accuracy Score: %f" % accuracy)

# precision = precision_score(y_true=y_test, y_pred=y_pred, average='micro')
# print("Precision Score: %f" % precision)

# recall = recall_score(y_true=y_test, y_pred=y_pred, average='micro')
# print("Recall Score: %f" % recall)

# f1 = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
# print('F1 Score: %f' % f1)

# # Calculate predicted probabilities
# y_pred_proba = model.predict_proba(X_test)

# # Keep only the proba for True
# y_pred_proba = y_pred_proba[:, 1]

# # Compute auc score
# auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)
# print('AUC Score: %f' % auc)

# # Produce classification Report
# print(classification_report(y_test, y_pred))

# # Plot Tree
# # Note: Plotting trees for multi-class problems can be complex and may not provide clear visualizations
# # You may want to visualize individual trees if you're dealing with a smaller number of classes
# class_names = df[dependent_variable].unique()
# plot_tree(model.estimators_[0], feature_names=independent_variables, class_names=class_names, filled=True)

#RANDOM FOREST 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize a Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=100)  # You can adjust the number of estimators as needed

X = df[independent_variables]
y = df[dependent_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Fit the model to the training data
model.fit(X_train, y_train)

# Now let's evaluate our model on the test set
y_pred = model.predict(X_test).round().astype(int)

# Evaluate the model
mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
print("Mean Squared Error: %f" % mse)

r2 = r2_score(y_true=y_test, y_pred=y_pred)
print("R-squared Score: %f" % r2)

## Eval Model 
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy Score: %f" % accuracy)


In [ ]:
print(y_pred)

In [ ]:
y_test

In [ ]:
# Now lets look at our feature importances
feature_imp = pd.Series(model.feature_importances_,index=independent_variables).sort_values(ascending=False)
feature_imp

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    # Add more hyperparameters to tune
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best hyperparameters to train the model
best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)


In [ ]:
feature_importances = best_model.feature_importances_
feature_importances

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Extract one decision tree from the random forest (e.g., the first tree)
one_tree = model.estimators_[0]

# Plot the decision tree
plt.figure(figsize=(20, 10))
plot_tree(one_tree, feature_names=independent_variables, class_names=class_names, filled=True)
plt.show()

## Sami trying here

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Your feature and target variable definitions
selected_features = ['Primary_Color_encoded', 'Sex_encoded', 'Intake_Condition_encoded', 'Intake_Type_encoded']
X = df[selected_features]
y = df['days_stayed']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define hyperparameter grid
params = {
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'max_leaf_nodes': [5, 10]
}

# Initialize GridSearchCV with RandomForestRegressor
grid_search_cv = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=params,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1
)

# Fit GridSearchCV with training data
grid_search_cv.fit(X_train, y_train)

# Get the best estimator based on mean squared error
best_model = grid_search_cv.best_estimator_

# Make predictions using the best model
y_pred = np.round(best_model.predict(X_test))

# Evaluate the model
mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
print("Mean Squared Error: %f" % mse)

r2 = r2_score(y_true=y_test, y_pred=y_pred)
print("R-squared Score: %f" % r2)

# Additional evaluation metrics if needed
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy Score: %f" % accuracy)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

# ... (your data loading and preprocessing code)
# Your feature and target variable definitions
selected_features = ['Primary_Color_encoded', 'Sex_encoded', 'Intake_Condition_encoded', 'Intake_Type_encoded']
X = df[selected_features]
y = df['days_stayed']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# New hyperparameter grid
params = {
    'n_estimators': [50, 100, 150, 200],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'max_leaf_nodes': [5, 10, 15]
}

# GridSearchCV with RandomForestRegressor
grid_search_cv_rf = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=params,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1
)

# GridSearchCV with GradientBoostingRegressor
grid_search_cv_gb = GridSearchCV(
    estimator=GradientBoostingRegressor(),
    param_grid=params,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1
)

# GridSearchCV with LinearRegression
grid_search_cv_lr = GridSearchCV(
    estimator=LinearRegression(),
    param_grid={},
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1
)

# GridSearchCV with SVR
grid_search_cv_svr = GridSearchCV(
    estimator=SVR(),
    param_grid={'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1
)

# Fit models
grid_search_cv_rf.fit(X_train, y_train)
grid_search_cv_gb.fit(X_train, y_train)
grid_search_cv_lr.fit(X_train, y_train)
grid_search_cv_svr.fit(X_train, y_train)

# Get best models
best_rf_model = grid_search_cv_rf.best_estimator_
best_gb_model = grid_search_cv_gb.best_estimator_
best_lr_model = grid_search_cv_lr.best_estimator_
best_svr_model = grid_search_cv_svr.best_estimator_

# Evaluate models
models = [best_rf_model, best_gb_model, best_lr_model, best_svr_model]
model_names = ['RandomForestRegressor', 'GradientBoostingRegressor', 'LinearRegression', 'SVR']

for model, name in zip(models, model_names):
    y_pred = np.round(model.predict(X_test))
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    r2 = r2_score(y_true=y_test, y_pred=y_pred)
    print(f"{name} - Mean Squared Error: {mse}, R-squared Score: {r2}")
